In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/9.part'):
    shutil.rmtree('./file/9.part')

os.makedirs('./file/9.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/8.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/8.url/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    try:
                        resp = requests.get(input_.loc[a, 'Url'],
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                type_ = html.xpath('//h1/span[@id="lblPartTypeName"]/text()')[0].strip()

                # = = = = = = = = = = = = = = =

                i, dict_src = 0, {}
                list_src = list(reversed([href.strip() for href in html.xpath('//a[@data-zoom-id="ProductPic"]/@href')]))
                for src in list_src:
                    if src.startswith('http'):
                        dict_src[str(i)] = src
                        i += 1

                # = = = = = = = = = = = = = = =

                part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =

                description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =

                application_summary = '\n'.join(html.xpath('//div[@id="divAppSummary"]/p/text()')).strip()
                
                # = = = = = = = = = = = = = = =

                brand = html.xpath('//span[@id="lblBrandName"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =

                vehicle_url = html.xpath('//input[@id="hidfDetailApp"]/@value')
                vehicle_url = f'''https://www.dormanproducts.com/{vehicle_url[0].strip()}''' if vehicle_url else ''

                # = = = = = = = = = = = = = = =

                dict_specification = {}
                list_tr = html.xpath('//section[@id="productSpec"]/div/table/tr')
                for i, tr in zip(range(len(list_tr)), list_tr):
                    dict_specification[str(i)] = {tr.xpath('./th/text()')[0].strip().removesuffix(':').strip(): tr.xpath('./td/text()')[0].strip()}

                # = = = = = = = = = = = = = = =

                list_oem = [text.strip() for text in html.xpath('//section[@id="productOE"]/div/table/tbody/tr/th/text()')]
                oem = ';'.join(sorted(list(set(list_oem)), key=list_oem.index))

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Type': type_,
                                         'Part Number': part_number,
                                         'Brand': brand,
                                         'Description': description,
                                         'Application Summary': application_summary,
                                         'OEM': oem,
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Vehicle Url': vehicle_url,
                                         'Json_Src': json.dumps(dict_src),
                                         'Json_Specification': json.dumps(dict_specification)}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(30):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/9.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/9.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/9.part/{file}',
              f'''./file/9.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：151

[状态：ok，尝试次数：1] - 100.00% - crawler_2222 > 17.https://www.dormanproducts.com/p-105780-712-280.aspx
[剩余数量：121] - [当前时间：18:06:37]

[状态：ok，尝试次数：1] - 100.00% - crawler_2222 > 9.https://www.dormanproducts.com/p-131671-713-475z.aspx
[剩余数量：120] - [当前时间：18:06:37]

[状态：ok，尝试次数：1] - 100.00% - crawler_2222 > 11.https://www.dormanproducts.com/p-106467-712-381.aspx
[剩余数量：119] - [当前时间：18:06:38]

[状态：ok，尝试次数：1] - 100.00% - crawler_2222 > 23.https://www.dormanproducts.com/p-131670-713-475t.aspx
[剩余数量：118] - [当前时间：18:06:38]

[状态：ok，尝试次数：1] - 100.00% - crawler_2222 > 10.https://www.dormanproducts.com/p-106475-712-681.aspx
[剩余数量：117] - [当前时间：18:06:38]

[状态：ok，尝试次数：1] - 100.00% - crawler_2222 > 18.https://www.dormanproducts.com/p-106453-712-281.aspx
[剩余数量：116] - [当前时间：18:06:39]

[状态：ok，尝试次数：1] - 100.00% - crawler_2222 > 7.https://www.dormanproducts.com/p-131656-713-375z.aspx
[剩余数量：115] - [当前时间：18:06:39]

[状态：ok，尝试次数：1] - 100.00% - crawler_2222 > 2.https://www.dormanproducts.com/p-126620-712-628.as

Progress: 100%|██████████████████████████████| 1/1 [00:00<00:00, 2792.48it/s]

Done ~
